# Heart Disease Prediction

In [ ]:
!pip install tensorflow==2.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 23.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: gast
    Found existing installation: gast 0.6.0
    Uninstalling gast-0.6.0:
      

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
print(tf.__version__)

2.13.0


In [ ]:
# Load the dataset
data = pd.read_csv('/data/cleaned_dataset.csv')

# Drop irrelevant columns
X = data.drop(['num', 'id', 'dataset'], axis=1)
y = (data['num'] > 0).astype(int)

# Encode binary columns
X['sex'] = X['sex'].map({'Male': 0, 'Female': 1})
X['fbs'] = X['fbs'].map({False: 0, True: 1})
X['exang'] = X['exang'].map({False: 0, True: 1})

# One-hot encode categorical columns
X = pd.get_dummies(X, columns=['cp', 'restecg', 'slope', 'thal'])

# Scale numerical columns
numerical_cols = ['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca']
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])


In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Define the model with more regularization and dropout
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],),kernel_regularizer=tf.keras.regularizers.l2(0.2)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model with a learning rate scheduler
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-4)  # Keep the learning rate as is
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC()])

In [ ]:
# Add callbacks for early stopping and learning rate scheduling
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

In [ ]:
# Ensure proper types and dimensions
X_train = np.array(X_train, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32)

# Check for and handle NaN or infinite values
X_train = np.nan_to_num(X_train)
y_train = np.nan_to_num(y_train)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=60,
    batch_size=64,
    validation_split=0.2,
    callbacks=[early_stopping, lr_scheduler]
)


Epoch 1/60
10/10 [==============================] - 2s 44ms/step - loss: 7.5040 - accuracy: 0.5850 - auc: 0.5949 - val_loss: 7.0537 - val_accuracy: 0.5646 - val_auc: 0.6605 - lr: 5.0000e-04
Epoch 2/60
10/10 [==============================] - 0s 9ms/step - loss: 7.1396 - accuracy: 0.6156 - auc: 0.6482 - val_loss: 6.7326 - val_accuracy: 0.6395 - val_auc: 0.7209 - lr: 5.0000e-04
Epoch 3/60
10/10 [==============================] - 0s 8ms/step - loss: 6.8859 - accuracy: 0.5884 - auc: 0.6278 - val_loss: 6.4263 - val_accuracy: 0.6667 - val_auc: 0.7703 - lr: 5.0000e-04
Epoch 4/60
10/10 [==============================] - 0s 38ms/step - loss: 6.4825 - accuracy: 0.6259 - auc: 0.6745 - val_loss: 6.1326 - val_accuracy: 0.6939 - val_auc: 0.7998 - lr: 5.0000e-04
Epoch 5/60
10/10 [==============================] - 0s 8ms/step - loss: 6.1491 - accuracy: 0.6854 - auc: 0.7273 - val_loss: 5.8504 - val_accuracy: 0.6939 - val_auc: 0.8157 - lr: 5.0000e-04
Epoch 6/60
10/10 [==============================] - 0

In [ ]:
# Evaluate the model
X_test = np.array(X_test, dtype=np.float32)
y_test = np.array(y_test, dtype=np.float32)
results = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {results[1]:.2f}")
print(f"Test AUC: {results[2]:.2f}")


6/6 [==============================] - 0s 4ms/step - loss: 0.6880 - accuracy: 0.8261 - auc: 0.8969
Test Accuracy: 0.83
Test AUC: 0.90


In [ ]:
# Save the model architecture to a JSON file
model_json = model.to_json()
with open("/content/model_architecture.json", "w") as json_file:
    json_file.write(model_json)
print("Model architecture saved as JSON.")

# Save the model weights to an HDF5 file
model.save_weights("/content/model.weights.h5")
print("Model weights saved separately.")



Model architecture saved as JSON.
Model weights saved separately.


In [ ]:
# Save the model as a Keras file
model.save("/content/heart_disease_model.keras")